In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import cv2
import random


def test_data_sanity():
    print("✅ from data.ipynb")

# RGB to class index mapping (used only by rgb_to_class_index)
COLOR_TO_CLASS = {
    (75, 25, 230): 0,       # BUILDING
    (180, 30, 145): 1,      # CLUTTER
    (75, 180, 60): 2,       # VEGETATION
    (48, 130, 245): 3,      # WATER
    (255, 255, 255): 4,     # GROUND
    (200, 130, 0): 5        # CAR
}


class StreamingDataGenerator(Sequence):
    def __init__(self, image_dir, elevation_dir, label_dir, batch_size=32, input_type='rgb', num_classes=6, shuffle=True):
        self.image_dir = image_dir
        self.elevation_dir = elevation_dir
        self.label_dir = label_dir
        self.batch_size = batch_size
        self.input_type = input_type
        self.shuffle = shuffle
        self.num_classes = num_classes
        self.tile_list = [f.replace('-ortho.png', '') for f in os.listdir(image_dir) if f.endswith('-ortho.png')]
        self.on_epoch_end()

    def __len__(self):
        return 1  # One batch per epoch

    def __getitem__(self, index):
        batch_x = []
        batch_y = []

        selected = np.random.choice(self.tile_list, self.batch_size, replace=False)

        for file_name in selected:
            try:
                # Load RGB
                rgb_path = os.path.join(self.image_dir, file_name + "-ortho.png")
                rgb_img = cv2.imread(rgb_path)
                if rgb_img is None:
                    print(f"❌ Skipping: Missing RGB at {rgb_path}")
                    continue
                rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)

                # Load elevation
                elev_path = os.path.join(self.elevation_dir, file_name + "-elev.npy")
                if not os.path.exists(elev_path):
                    print(f"❌ Skipping: Missing elevation at {elev_path}")
                    continue
                elev_img = np.load(elev_path)
                if elev_img.ndim == 2:
                    elev_img = np.expand_dims(elev_img, axis=-1)

                # Load label
                label_path = os.path.join(self.label_dir, file_name + "-label.png")
                label_img = cv2.imread(label_path)
                if label_img is None:
                    print(f"❌ Skipping: Missing label at {label_path}")
                    continue
                label_img = cv2.cvtColor(label_img, cv2.COLOR_BGR2RGB)

                label = self.rgb_to_class_index(label_img)
                label = tf.keras.utils.to_categorical(label, num_classes=self.num_classes)

                # Construct input
                if self.input_type == '1ch':
                    merged = np.expand_dims(cv2.cvtColor(rgb_img, cv2.COLOR_RGB2GRAY), axis=-1)
                elif self.input_type == '2ch':
                    grayscale = np.expand_dims(cv2.cvtColor(rgb_img, cv2.COLOR_RGB2GRAY), axis=-1)
                    merged = np.concatenate([grayscale, elev_img], axis=-1)
                elif self.input_type == 'rgb':
                    merged = rgb_img
                elif self.input_type == 'rgb_elevation':
                    merged = np.concatenate([rgb_img, elev_img], axis=-1)
                else:
                    raise ValueError(f"Invalid input_type: {self.input_type}")

                batch_x.append(merged / 255.0)
                batch_y.append(label)

            except Exception as e:
                print(f"❌ Error processing tile {file_name}: {str(e)}")
                continue

        if len(batch_x) == 0:
            print("⚠️ WARNING: Generator yielded an empty batch. Returning dummy zero batch.")
            dummy_x = np.zeros((1, 512, 512, 4), dtype=np.float32)
            dummy_y = np.zeros((1, 512, 512, self.num_classes), dtype=np.float32)
            return dummy_x, dummy_y

        return np.array(batch_x, dtype=np.float32), np.array(batch_y, dtype=np.float32)

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.tile_list)

    def rgb_to_class_index(self, label_rgb):
        h, w, _ = label_rgb.shape
        label = np.full((h, w), -1, dtype=np.int32)

        for color, idx in COLOR_TO_CLASS.items():
            mask = np.all(label_rgb == color, axis=-1)
            label[mask] = idx

        if np.any(label == -1):
            raise ValueError("❌ Unknown RGB values in label mask.")

        return label
